In [ ]:
# reference from tensorflow for machine intelligence

In [51]:
import tensorflow as tf
import numpy as np

In [52]:
# Create a Graph to start (note: this is not necessary)
mygraph= tf.Graph()

### Building the Graph

在这个例子中 我们需要构建5块内容：

1）variable： 用于存储两个变量
   训练次数 （不参与训练）
   output
   
2）transformation： 对输入参数进行变形
   采用 placeholder

3）update： 更新的规律

4）summary 用于 tensor board scalars

5）global_operations 
   初始化参数
   merge summary

In [53]:
# Set the graph we are working on
with mygraph.as_default():
    
    # This name scope contains varibales not for trainning
    with tf.name_scope('variables'):
        # Variable indicating how many times the graph run
        global_step = tf.Variable (0, dtype=tf.int32, trainable= False, name = 'global_step')
        
        # Variable of sum of outputs
        total_outputs= tf.Variable (0.0, dtype=tf.float32, trainable= False, name = 'total_output')
    
    # This name scope contains varibales for transformation
    with tf.name_scope('transformation'):
        
        # input name scope
        with tf.name_scope('input'):
            a= tf.placeholder(tf.float32,shape=(None), name= 'input_placeholder_a')
        
        # Middle name scope
        with tf.name_scope('middle'):
            b= tf.reduce_prod(a, name='production_b')
            c= tf.reduce_sum(a, name='sum_c')
        # output name scope
        with tf.name_scope('output'):
            output=tf.add(b,c, name='output')
            
    # This name scope used to update variables   
    with tf.name_scope('update'):
        # add output to total_output
        update_total= total_outputs.assign_add (output)
        total_step= global_step.assign_add(1)
        
    # This name scope create tensor board summaries 
    with tf.name_scope('summaries'):
        average= tf.div(update_total, tf.cast(total_step,tf.float32), name= 'average')
        tf.summary.scalar ('output_summary',output)
        tf.summary.scalar ('total_summary',update_total)
        tf.summary.scalar ('average_summary',average)
        
    with tf.name_scope('global_operations'):
        # Initializes
        init=tf.global_variables_initializer()
        # Merge all summaries into one operation
        merged_summaries = tf.summary.merge_all()
        


### Running the Graph : Session

In [54]:
# Start a session, 指定我们之前建的 graph
sess=tf.Session(graph=mygraph)
# open a summary writer
summary_writer=tf.summary.FileWriter('./example_1', sess.graph)

In [55]:
sess.run(init)

In [56]:
def run_graph(input_tensor):
    feed_dict={a: input_tensor}
    outputs, step, summary = sess.run([output, total_step, merged_summaries], feed_dict=feed_dict)
   
    print(outputs)
    print(step)
    
    summary_writer.add_summary(summary, global_step=step)
    summary_writer.add_graph(sess.graph)

In [57]:
run_graph([2,8])
run_graph([3,1,1])
run_graph([5])
run_graph([2,5,9,9])
run_graph([1,0,0])
run_graph([[1],[0],[0]])

26.0
1
8.0
2
10.0
3
835.0
4
1.0
5
1.0
6


In [44]:
# Call this method to make sure that all pending events have been written to disk.
summary_writer.flush()

In [45]:
summary_writer.close()
sess.close()

In [58]:
!tensorboard --logdir=./example_1

'tensorboard' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
